### Structured output
Models can be requested to provide their response in a format matching a given schema. This is useful for ensuring the output can be easily parsed and used in subsequent processing. LangChain supports multiple schema types and methods for enforcing structured output.

### Pydantic
Pydantic models provide the richest feature set with field validation, descriptions, and nested structures.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [10]:
from langchain.chat_models import init_chat_model
model = init_chat_model(model="groq:llama-3.3-70b-versatile")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000198B793E490>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000198B793EE90>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
response = model.invoke("Hello, how are you?")
response.content

"Hello. I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to assist you. How can I help you today?"

In [14]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    title: str = Field(description="The title of the movie")
    year: int = Field(description="The year of release of the movie")
    director: str = Field(description="Name of the director of the movie")
    rating: float = Field(description="The rating of the movie out of 10")

In [15]:
model_with_structure = model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000198B793E490>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000198B793EE90>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'The year of release of the movie', 'type': 'integer'}, 'director': {'description': 'Name of the director of the movie', 'type': 'string'}, 'rating': {'description': 'The rating of the movie out of 10', 'type': 'number'}}, 'requir

In [18]:
response = model.invoke("Provide details of the movie Inception")
print(response)

content='Inception is a 2010 science fiction action film written, co-produced, and directed by Christopher Nolan. The film features an ensemble cast, including Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy, Ken Watanabe, Dileep Rao, Cillian Murphy, Tom Berenger, and Marion Cotillard.\n\n**Plot:**\n\nThe movie revolves around Cobb (Leonardo DiCaprio), a skilled thief who specializes in entering people\'s dreams and stealing their secrets. Cobb is hired by a wealthy businessman named Saito (Ken Watanabe) to perform a task known as "inception" - planting an idea in someone\'s mind instead of stealing one.\n\nSaito wants Cobb to convince Robert Fischer (Cillian Murphy), the son of a dying business magnate, to dissolve his father\'s company. In return, Saito promises to clear Cobb\'s name, which is wanted by the authorities, and allow him to return to the United States to see his children.\n\nCobb assembles a team of experts to help him with the task: Arthur (Joseph Gordon-

In [19]:
response = model_with_structure.invoke("Provide details of the movie Inception")
print(response)

title='Inception' year=2010 director='Christopher Nolan' rating=8.5


### Message Output alongside Parsed Structure

In [20]:
class Movie(BaseModel):
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year of release of the movie")
    director: str = Field(..., description="The name of the director of the movie")
    rating: float = Field(..., description="The rating of the movie out of 10")

In [21]:
model_with_structure = model.with_structured_output(Movie, include_raw=True)
model_with_structure

{
  raw: RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000198B793E490>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000198B793EE90>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'The year of release of the movie', 'type': 'integer'}, 'director': {'description': 'The name of the director of the movie', 'type': 'string'}, 'rating': {'description': 'The rating of the movie out of 10', 'type': 'numbe

In [24]:
response = model_with_structure.invoke("Provide details about the movie Inception")
response

{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'g3wmh1jv5', 'function': {'arguments': '{"director":"Christopher Nolan","rating":8.5,"title":"Inception","year":2010}', 'name': 'Movie'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 285, 'total_tokens': 317, 'completion_time': 0.053211353, 'completion_tokens_details': None, 'prompt_time': 0.014050302, 'prompt_tokens_details': None, 'queue_time': 0.056080948, 'total_time': 0.067261655}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd773-f018-79d3-8cb1-d6c25a74d82a-0', tool_calls=[{'name': 'Movie', 'args': {'director': 'Christopher Nolan', 'rating': 8.5, 'title': 'Inception', 'year': 2010}, 'id': 'g3wmh1jv5', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 285, 'output_tokens': 32,

### Nested Structured Output

In [27]:
class Actor(BaseModel):
    name: str
    role: str

class MovieDetails(BaseModel):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description="The budget of the movie in millions USD")

In [28]:
model_with_structure = model.with_structured_output(MovieDetails)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000198B793E490>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000198B793EE90>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'MovieDetails', 'description': '', 'parameters': {'properties': {'title': {'type': 'string'}, 'year': {'type': 'integer'}, 'cast': {'items': {'properties': {'name': {'type': 'string'}, 'role': {'type': 'string'}}, 'required': ['name', 'role'], 'type': 'object'}, 'type': 'array'}, 'genres': {'items': {'type': 'string'}, 'type': 'array'}, 'budget': {'anyOf': [{'type': 'number'}, {'

In [30]:
response = model_with_structure.invoke("Provide the details about the movie Inception")
print(response)

title='Inception' year=2010 cast=[Actor(name='Leonardo DiCaprio', role='Cobb'), Actor(name='Joseph Gordon-Levitt', role='Arthur')] genres=['Action', 'Sci-Fi'] budget=160.0


### TypedDict
TypedDict provides a simpler alternative using Python’s built-in typing, ideal when you don’t need runtime validation.

In [31]:
from typing_extensions import TypedDict, Annotated

class MovieDict(TypedDict):
    """A movie with details"""
    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The release year of the movie"]
    director: Annotated[str, ..., "The name of the director of the movie"]
    rating: Annotated[float, ..., "The rating of the movie out of 10"]

In [32]:
model_with_typeddict_structure = model.with_structured_output(MovieDict)
model_with_typeddict_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000198B793E490>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000198B793EE90>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'MovieDict', 'description': 'A movie with details', 'parameters': {'type': 'object', 'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'The release year of the movie', 'type': 'integer'}, 'director': {'description': 'The name of the director of the movie', 'type': 'string'}, 'rating': {'description': 'The rating of the mo

In [35]:
response = model_with_typeddict_structure.invoke("Provide me details about the movie Tenet")
response

{'director': 'Christopher Nolan',
 'rating': 7.4,
 'title': 'Tenet',
 'year': 2020}

In [36]:
class Actor(TypedDict):
    name: Annotated[str, ..., "The name of the actor"]
    role: Annotated[str, ..., "The role played by the actor"]

class MovieDetails(TypedDict):
    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The release year of the movie"]
    cast: Annotated[list[Actor], ..., "The list of actors in the movie"]
    director: Annotated[str, ..., "The director of the movie"]
    rating: Annotated[float, ..., "The rating of the movie"]
    budget: Annotated[float, ..., "The budget of the movie in Millions USD"]

In [37]:
model_with_nested_typeddict_structure = model.with_structured_output(MovieDetails)
model_with_nested_typeddict_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000198B793E490>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000198B793EE90>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'MovieDetails', 'description': "dict() -> new empty dictionary\ndict(mapping) -> new dictionary initialized from a mapping object's\n    (key, value) pairs\ndict(iterable) -> new dictionary initialized as if via:\n    d = {}\n    for k, v in iterable:\n        d[k] = v\ndict(**kwargs) -> new dictionary initialized with the name=value pairs\n    in the keyword argument list.  For 

In [38]:
response = model_with_nested_typeddict_structure.invoke("Provide me some details about the movie Tenet")
response

{'budget': 200,
 'cast': [{'name': 'John David Washington', 'role': 'The Protagonist'},
  {'name': 'Robert Pattinson', 'role': 'Neil'}],
 'director': 'Christopher Nolan',
 'rating': 7.1,
 'title': 'Tenet',
 'year': 2020}

### DataClasses
A data class is a class typically containing mainly data, although there aren’t really any restrictions. You create it using the @dataclass decorator

In [ ]:
class ContactDetails(BaseModel):
    name: str = Field(..., description="The name of the person")
    email: str = Field(..., description="The email address of the person")
    phone: str = Field(..., description="The phone number of the person")

from langchain.agents import create_agent

agent = create_agent(
    model="groq:llama-3.3-70b-versatile",
    response_format=ContactDetails
)

result = agent.invoke({
    "messages" : [{"role" : "user", "content" : "Extract contact details from: Gaurvit Kumar, kumargaurvit@gmail.com, 8448856789"}]
})

result["structured_response"]

ContactDetails(name='Gaurvit Kumar', email='kumargaurvit@gmail.com', phone='8448856789')

In [46]:
from typing_extensions import TypedDict, Annotated
class ContactDetails(TypedDict):
    name: Annotated[str, Field(..., description="The name of the person")]
    email: Annotated[str, Field(..., description="The email address of the person")]
    phone: Annotated[str, Field(..., description="The phone number of the person")]

from langchain.agents import create_agent

agent = create_agent(
    model="groq:llama-3.3-70b-versatile",
    response_format=ContactDetails
)

result = agent.invoke({
    "messages" : [{"role" : "user", "content" : "Extract contact details from: Gaurvit Kumar, kumargaurvit@gmail.com, 8448856789"}]
})

result["structured_response"]

{'name': 'Gaurvit Kumar',
 'email': 'kumargaurvit@gmail.com',
 'phone': '8448856789'}

In [49]:
from typing_extensions import Annotated
from dataclasses import dataclass

@dataclass
class ContactDetails:
    """Contact Details of a person"""
    name: Annotated[str, Field(..., description="The name of the person")]
    email: Annotated[str, Field(..., description="The email address of the person")]
    phone: Annotated[str, Field(..., description="The phone number of the person")]

from langchain.agents import create_agent

agent = create_agent(
    model="groq:llama-3.3-70b-versatile",
    response_format=ContactDetails
)

result = agent.invoke({
    "messages" : [{"role" : "user", "content" : "Extract contact details from: Gaurvit Kumar, kumargaurvit@gmail.com, 8448856789"}]
})

result["structured_response"]

ContactDetails(name='Gaurvit Kumar', email='kumargaurvit@gmail.com', phone='8448856789')